<a href="https://colab.research.google.com/github/sanghaimuskan/Autocorrect-Systems/blob/master/AutocorrectSystems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 1: Data Preprocessing**

# **Step 1: Opening file and extracting words and converting to lowercase**




In [98]:
import re
import pandas as pd
import numpy as np
from collections import Counter

In [99]:
def process_data(file):
  words =[]

  with open(file) as f:
    file_data = f.read()
  
  file_data = file_data.lower()
  words = re.findall('\w+', file_data)

  return words

In [100]:
!pip install -U -q PyDrive

In [101]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [102]:
link = 'https://drive.google.com/file/d/1Q07I_80HJ95Y7jTNsKXiu30VjfUpXOFM/view?usp=sharing'

In [103]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

sharing


In [104]:
word_l = process_data('/content/drive/My Drive/Colab Notebooks/corpus.txt') 
vocab = set(word_l)  # this will be your new vocabulary

In [105]:
print(f"The first ten words in the text are: \n{word_l[0:10]}")

The first ten words in the text are: 
['o', 'for', 'a', 'muse', 'of', 'fire', 'that', 'would', 'ascend', 'the']


# **Step2: Getting the Word Count**

In [106]:
def word_count(word_l):
  wordcount = {}

  for i in word_l:
    wordcount = Counter(word_l)
  return wordcount

In [107]:
word_count_dict = word_count(word_l)

print(f"The count for the word 'playing' is {word_count_dict.get('playing',0)}")

The count for the word 'playing' is 1


# **Step3: Getting Probabilities of each word**

In [108]:
def get_probs(word_count_dict):

  total = sum(word_count_dict.values())

  probs = {}
  for key in word_count_dict.keys():
    probs[key] = word_count_dict[key]/total
  
  return probs

In [109]:
probs = get_probs(word_count_dict)
print(f"P('am') is {probs['am']:.4f}")

P('am') is 0.0023


# **Part2: String Manipulation**

# **Step1: Deleting a letter to make the word correct**

In [110]:
def delete_letter(word):

  split =[]
  del_word = []

  for c in range(len(word)):
    split.append((word[:c], word[c:]))
  
  for a ,b in split:
    del_word.append(a[:]+b[1:])

  return del_word

In [111]:
word = 'Susn'

delete_letter(word)

['usn', 'Ssn', 'Sun', 'Sus']

# **Step 2: Switching between letters in a word**

In [112]:
def switch(word):
  split =[]
  switch = []

  for c in range(len(word)):
    split.append((word[:c], word[c:]))
  
  for a,b in split:
    if(len(b)>2):
      switch.append(a+b[1]+b[0]+b[2:])
  
  return switch

In [113]:
switch("maen")

['amen', 'mean']

# **Step3: Replacing letter**

In [115]:
def replace_letter(word):
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_l = []
    split_l = []
    
   
    for c in range(len(word)):
        split_l.append((word[0:c],word[c:]))
    replace_l = [a + l + (b[1:] if len(b)> 1 else '') for a,b in split_l if b for l in letters]
    replace_set=set(replace_l)      
    
    # turn the set back into a list and sort it, for easier viewing
    replace_l = sorted(list(replace_set)) 
    replace_l = set(replace_l)   
    
    return replace_l

In [116]:
replace_letter('msn')

{'asn',
 'bsn',
 'csn',
 'dsn',
 'esn',
 'fsn',
 'gsn',
 'hsn',
 'isn',
 'jsn',
 'ksn',
 'lsn',
 'man',
 'mbn',
 'mcn',
 'mdn',
 'men',
 'mfn',
 'mgn',
 'mhn',
 'min',
 'mjn',
 'mkn',
 'mln',
 'mmn',
 'mnn',
 'mon',
 'mpn',
 'mqn',
 'mrn',
 'msa',
 'msb',
 'msc',
 'msd',
 'mse',
 'msf',
 'msg',
 'msh',
 'msi',
 'msj',
 'msk',
 'msl',
 'msm',
 'msn',
 'mso',
 'msp',
 'msq',
 'msr',
 'mss',
 'mst',
 'msu',
 'msv',
 'msw',
 'msx',
 'msy',
 'msz',
 'mtn',
 'mun',
 'mvn',
 'mwn',
 'mxn',
 'myn',
 'mzn',
 'nsn',
 'osn',
 'psn',
 'qsn',
 'rsn',
 'ssn',
 'tsn',
 'usn',
 'vsn',
 'wsn',
 'xsn',
 'ysn',
 'zsn'}

# **Step4:Insert Letter**

In [117]:
def insert_letter(word):
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []
    
    for c in range(len(word)+1):
        split_l.append((word[0:c],word[c:]))
    insert_l = [ a + l + b for a,b in split_l for l in letters]
     
    return insert_l

In [118]:
insert_l = insert_letter('at')

# **Part 3: Combining the edits**

# Step 1: 1 letter edit

In [121]:
def edit_one_letter(word):
    
    edit_one_set = set()
    
    edit_one_set.update(delete_letter(word))
    edit_one_set.update(switch(word))
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))

    return edit_one_set

In [122]:
edit_one_letter('mp')

{'amp',
 'ap',
 'bmp',
 'bp',
 'cmp',
 'cp',
 'dmp',
 'dp',
 'emp',
 'ep',
 'fmp',
 'fp',
 'gmp',
 'gp',
 'hmp',
 'hp',
 'imp',
 'ip',
 'jmp',
 'jp',
 'kmp',
 'kp',
 'lmp',
 'lp',
 'm',
 'ma',
 'map',
 'mb',
 'mbp',
 'mc',
 'mcp',
 'md',
 'mdp',
 'me',
 'mep',
 'mf',
 'mfp',
 'mg',
 'mgp',
 'mh',
 'mhp',
 'mi',
 'mip',
 'mj',
 'mjp',
 'mk',
 'mkp',
 'ml',
 'mlp',
 'mm',
 'mmp',
 'mn',
 'mnp',
 'mo',
 'mop',
 'mp',
 'mpa',
 'mpb',
 'mpc',
 'mpd',
 'mpe',
 'mpf',
 'mpg',
 'mph',
 'mpi',
 'mpj',
 'mpk',
 'mpl',
 'mpm',
 'mpn',
 'mpo',
 'mpp',
 'mpq',
 'mpr',
 'mps',
 'mpt',
 'mpu',
 'mpv',
 'mpw',
 'mpx',
 'mpy',
 'mpz',
 'mq',
 'mqp',
 'mr',
 'mrp',
 'ms',
 'msp',
 'mt',
 'mtp',
 'mu',
 'mup',
 'mv',
 'mvp',
 'mw',
 'mwp',
 'mx',
 'mxp',
 'my',
 'myp',
 'mz',
 'mzp',
 'nmp',
 'np',
 'omp',
 'op',
 'p',
 'pmp',
 'pp',
 'qmp',
 'qp',
 'rmp',
 'rp',
 'smp',
 'sp',
 'tmp',
 'tp',
 'ump',
 'up',
 'vmp',
 'vp',
 'wmp',
 'wp',
 'xmp',
 'xp',
 'ymp',
 'yp',
 'zmp',
 'zp'}

# **Step 2: Edit 2 letter**

In [123]:
def edit_two_letters(word):    
    edit_two_set = set()
    
    edit_one = edit_one_letter(word)
    for w in edit_one:
        if w:
            edit_two = edit_one_letter(w)
            edit_two_set.update(edit_two)

    
    return edit_two_set

In [127]:
edit = edit_two_letters("a")
print(sorted(list(edit))[:10])

['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']


# **Part 4: suggest spelling suggestions**

In [128]:
def get_corrections(word, probs, vocab):
    
    suggestions = []
    n_best = []
    
    suggestions = list((word in vocab and word) or edit_one_letter(word).intersection(vocab) or edit_two_letters(word).intersection(vocab))
    n_best = [[s,probs[s]] for s in list(reversed(suggestions))]
    
    

    return n_best

In [129]:
my_word = 'dys' 
get_corrections(my_word, probs, vocab)

[['days', 0.0004103405826836274], ['dye', 1.865184466743761e-05]]

# **Part 4: Minimum Edit Distance**

Minimum cost to edit a word
# **Step 1: Dynnamic Programming**

In [130]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
   
    # use deletion and insert cost as  1
    m = len(source) 
    n = len(target) 
    #initialize cost matrix with zeros and dimensions (m+1,n+1) 
    D = np.zeros((m+1, n+1), dtype=int) 
    
    # Fill in column 0, from row 1 to row m, both inclusive
    for row in range(1,m+1): # Replace None with the proper range
        D[row,0] = D[row-1,0] + del_cost
        
    # Fill in row 0, for all columns from 1 to n, both inclusive
    for col in range(1, n+1): # Replace None with the proper range
        D[0,col] = D[0,col-1] + ins_cost
        
    # Loop through row 1 to row m, both inclusive
    for row in range(1,m+1): 
        
        # Loop through column 1 to column n, both inclusive
        for col in range(1,n+1):
            
            # Intialize r_cost to the 'replace' cost that is passed into this function
            r_cost = rep_cost
            
            # Check to see if source character at the previous row
            # matches the target character at the previous column, 
            if source[row-1] == target[col-1]:
                # Update the replacement cost to 0 if source and target are the same
                r_cost = 0
                
            # Update the cost at row, col based on previous entries in the cost matrix
            # Refer to the equation calculate for D[i,j] (the minimum of three calculated costs)
            D[row,col] = min([D[row-1,col]+del_cost, D[row,col-1]+ins_cost, D[row-1,col-1]+r_cost])
          
    # Set the minimum edit distance with the cost found at row m, column n
    med = D[m,n]
    
    return D, med

In [131]:
source =  'play'
target = 'stay'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  4 

   #  s  t  a  y
#  0  1  2  3  4
p  1  2  3  4  5
l  2  3  4  5  6
a  3  4  5  4  5
y  4  5  6  5  4
